In [8]:
import random
import json
import os
import yaml
from tqdm.notebook import tqdm
import pandas as pd
def process_datasets(config_folder: str, output_folder: str):
    """
    遍历给定文件夹内的 YAML 配置文件，处理相应的 JSON 文件，并生成 JSONL 格式的输出。

    Args:
        config_folder (str): 包含 YAML 配置文件的文件夹路径。
        output_folder (str): 用于保存新生成的 JSONL 文件的文件夹路径。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    for root, _, files in os.walk(config_folder):
        for file_name in files:
            if file_name.endswith('.yaml') and 'config_' in file_name:
                yaml_path = os.path.join(root, file_name)
                
                print(f"\nProcessing YAML file: {yaml_path}")
                try:
                    with open(yaml_path, 'r', encoding='utf-8') as f:
                        yaml_data = yaml.safe_load(f)
                except Exception as e:
                    print(f"Error reading YAML file {yaml_path}: {e}")
                    continue

                if 'train_file_path' in yaml_data:
                    json_path = yaml_data['train_file_path']
                    dataset_name = yaml_data.get('dataset', 'unknown_dataset')
                    task_name = yaml_data.get('task', 'unknown_task')
                    
                    full_dataset_name = f"{task_name}_{dataset_name}"
                    
                    if not os.path.exists(json_path):
                        print(f"Warning: JSON file not found at {json_path}. Skipping.")
                        continue
                        
                    print(f"  -> Reading JSON file: {json_path}")
                    try:
                        df = pd.read_json(json_path)
                    except Exception as e:
                        print(f"Error reading JSON file {json_path}: {e}")
                        continue
                    
                    # 检查所需的列是否存在
                    if 'instruction' not in df.columns or 'output' not in df.columns:
                        print(f"Warning: JSON file {json_path} is missing 'instruction' or 'output' column. Skipping.")
                        continue
                        
                    # 创建新的 JSONL 文件名
                    full_output_file = os.path.join(output_folder, f"{full_dataset_name}_full.jsonl")
                    subset_output_file = os.path.join(output_folder, f"{full_dataset_name}_subset.jsonl")
                    
                    # 生成完整数据集
                    full_data_list = []
                    for index, row in tqdm(df.iterrows(), total=len(df), desc="  -> Formatting data"):
                        new_entry = {
                            "dataset": full_dataset_name,
                            "id": f"{full_dataset_name}_{index}",
                            "messages": [
                                {
                                    "role": "user",
                                    "content": row['instruction']
                                },
                                {
                                    "role": "assistant",
                                    "content": row['output']
                                }
                            ]
                        }
                        full_data_list.append(new_entry)
                    
                    # 保存完整数据集
                    with open(full_output_file, 'w', encoding='utf-8') as f:
                        for item in full_data_list:
                            f.write(json.dumps(item, ensure_ascii=False) + '\n')
                    print(f"  -> Successfully created full JSONL file: {full_output_file}")
                    
                    # 随机抽取1%的数据样本
                    sample_size = max(1, int(len(full_data_list) * 0.3))
                    subset_data_list = random.sample(full_data_list, sample_size)
                    
                    # 保存子集
                    with open(subset_output_file, 'w', encoding='utf-8') as f:
                        for item in subset_data_list:
                            f.write(json.dumps(item, ensure_ascii=False) + '\n')
                    print(f"  -> Successfully created 1% subset JSONL file: {subset_output_file}")
def merge_jsonl_files(input_folder: str, output_folder: str):
    """
    将指定文件夹内的所有 full.jsonl 和 subset.jsonl 文件合并。

    Args:
        input_folder (str): 包含要合并的 JSONL 文件的文件夹路径。
        output_folder (str): 用于保存合并后的文件的文件夹路径。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    # 合并 full.jsonl 文件
    full_output_path = os.path.join(output_folder, 'full.jsonl')
    full_file_list = [f for f in os.listdir(input_folder) if f.endswith('_full.jsonl')]
    
    if not full_file_list:
        print("No files ending with '_full.jsonl' were found. Skipping full file merge.")
    else:
        print(f"Found {len(full_file_list)} full files to merge.")
        with open(full_output_path, 'w', encoding='utf-8') as outfile:
            for file_name in full_file_list:
                file_path = os.path.join(input_folder, file_name)
                print(f"  -> Merging: {file_name}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    for line in infile:
                        # 逐行读取并写入到新的文件中
                        outfile.write(line)
        print(f"Successfully merged all full files into: {full_output_path}")
        
    # ---

    # 合并 subset.jsonl 文件
    subset_output_path = os.path.join(output_folder, 'subset.jsonl')
    subset_file_list = [f for f in os.listdir(input_folder) if f.endswith('_subset.jsonl')]

    if not subset_file_list:
        print("No files ending with '_subset.jsonl' were found. Skipping subset file merge.")
    else:
        print(f"Found {len(subset_file_list)} subset files to merge.")
        with open(subset_output_path, 'w', encoding='utf-8') as outfile:
            for file_name in subset_file_list:
                file_path = os.path.join(input_folder, file_name)
                print(f"  -> Merging: {file_name}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    for line in infile:
                        # 逐行读取并写入到新的文件中
                        outfile.write(line)
        print(f"Successfully merged all subset files into: {subset_output_path}")

import os
import yaml
import pandas as pd
import json
import random
from tqdm import tqdm

def process_datasets_test(config_folder: str, output_folder: str):
    """
    Traverses YAML configuration files in a given folder, processes the corresponding
    JSON test files, and generates a 1% random subset in JSONL format,
    following the 'task-dataset-test-subset.jsonl' naming convention.

    Args:
        config_folder (str): The path to the folder containing the YAML configuration files.
        output_folder (str): The path to the folder where the new JSONL files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    for root, _, files in os.walk(config_folder):
        for file_name in files:
            if file_name.endswith('.yaml') and 'config_' in file_name:
                yaml_path = os.path.join(root, file_name)

                print(f"\nProcessing YAML file: {yaml_path}")
                try:
                    with open(yaml_path, 'r', encoding='utf-8') as f:
                        yaml_data = yaml.safe_load(f)
                except Exception as e:
                    print(f"Error reading YAML file {yaml_path}: {e}")
                    continue

                if 'test_file_path' in yaml_data:
                    json_path = yaml_data['test_file_path']
                    dataset_name = yaml_data.get('dataset', 'unknown_dataset')
                    task_name = yaml_data.get('task', 'unknown_task')

                    # Create a unified dataset name from task and dataset
                    unified_dataset_name = f"{task_name}_{dataset_name}"
                    
                    if not os.path.exists(json_path):
                        print(f"Warning: JSON file not found at {json_path}. Skipping.")
                        continue

                    print(f"  -> Reading JSON file: {json_path}")
                    try:
                        df = pd.read_json(json_path)
                    except Exception as e:
                        print(f"Error reading JSON file {json_path}: {e}")
                        continue

                    # Check for required columns
                    if 'instruction' not in df.columns or 'output' not in df.columns:
                        print(f"Warning: JSON file {json_path} is missing 'instruction' or 'output' column. Skipping.")
                        continue

                    # Generate the output file name according to the specified standard
                    output_file_name = f"{task_name}-{dataset_name}-test-subset.jsonl"
                    output_path = os.path.join(output_folder, output_file_name)

                    # Generate the full data list for sampling
                    full_data_list = []
                    for index, row in tqdm(df.iterrows(), total=len(df), desc="  -> Formatting data"):
                        new_entry = {
                            "dataset": unified_dataset_name,
                            "id": f"{unified_dataset_name}_{index}",
                            "messages": [
                                {
                                    "role": "user",
                                    "content": row['instruction']
                                },
                                {
                                    "role": "assistant",
                                    "content": row['output']
                                }
                            ]
                        }
                        full_data_list.append(new_entry)

                    # Randomly sample 1% of the data
                    if not full_data_list:
                        print(f"Warning: No data found in {json_path}. Skipping subset creation.")
                        continue

                    sample_size = max(1, int(len(full_data_list) * 0.01))
                    subset_data_list = random.sample(full_data_list, sample_size)

                    # Save the subset to the new file
                    with open(output_path, 'w', encoding='utf-8') as f:
                        for item in subset_data_list:
                            f.write(json.dumps(item, ensure_ascii=False) + '\n')
                    print(f"  -> Successfully created 1% subset JSONL file: {output_path}")

In [9]:
process_datasets_test('script/','../LESS/LESS_data/')


Processing YAML file: script/config_CTA_WebTable.yaml
  -> Reading JSON file: train/CTA/WebTable/WebTable-test.json


  -> Formatting data: 100%|██████████| 17709/17709 [00:00<00:00, 26900.04it/s]


  -> Successfully created 1% subset JSONL file: ../LESS/LESS_data/CTA-WebTable-test-subset.jsonl

Processing YAML file: script/config_RE_RE.yaml
  -> Reading JSON file: train/RE/RE/RE-test.json


  -> Formatting data: 100%|██████████| 2072/2072 [00:00<00:00, 28819.95it/s]


  -> Successfully created 1% subset JSONL file: ../LESS/LESS_data/RE-RE-test-subset.jsonl

Processing YAML file: script/config_ER_wdc.yaml
  -> Reading JSON file: train/ER/wdc/wdc-test.json


...

  -> Successfully created 1% subset JSONL file: ../LESS/LESS_data/SM-CMS-test-subset.jsonl

Processing YAML file: script/config_AVE_oa_mine.yaml
  -> Reading JSON file: train/AVE/oa_mine/oa_mine-test.json


  -> Formatting data: 100%|██████████| 2451/2451 [00:00<00:00, 28955.16it/s]


  -> Successfully created 1% subset JSONL file: ../LESS/LESS_data/AVE-oa_mine-test-subset.jsonl

Processing YAML file: script/config_CTA_SimTab.yaml
  -> Reading JSON file: /data/home/wangys/MELD/dataset/CTA/SimTab_test_few.json


  -> Formatting data: 100%|██████████| 7610/7610 [00:00<00:00, 29142.80it/s]

  -> Successfully created 1% subset JSONL file: ../LESS/LESS_data/CTA-SimTab-test-subset.jsonl


In [4]:
pd.read_json('../LESS/LESS_data/subset.jsonl',lines=True)

,dataset,id,messages
0,CTA_WebTable,CTA_WebTable_6687,"[{'role': 'user', 'content': 'You are an exper..."
1,CTA_WebTable,CTA_WebTable_1554,"[{'role': 'user', 'content': 'You are an exper..."
2,CTA_WebTable,CTA_WebTable_7171,"[{'role': 'user', 'content': 'You are an exper..."
3,CTA_WebTable,CTA_WebTable_14472,"[{'role': 'user', 'content': 'You are an exper..."
4,CTA_WebTable,CTA_WebTable_349,"[{'role': 'user', 'content': 'You are an exper..."
...,...,...,...
29921,CTA_SimTab,CTA_SimTab_6183,"[{'role': 'user', 'content': 'You are an exper..."
29922,CTA_SimTab,CTA_SimTab_3096,"[{'role': 'user', 'content': 'You are an exper..."
29923,CTA_SimTab,CTA_SimTab_3508,"[{'role': 'user', 'content': 'You are an exper..."
29924,CTA_SimTab,CTA_SimTab_5772,"[{'role': 'user', 'content': 'You are an exper..."


In [6]:
import numpy as np
time_dict = np.load('Influence_single/CTA/WebTable/time.npy',allow_pickle=True).item()
time_dict

{'gradient-calculation': 1577.7494127750397, 'IF-Score': 241.13814163208008}

In [1]:
import torch
result = torch.load('../LESS/LESS_output/CTA_SimTab/influence_score.pt')
result.shape

torch.Size([29926])

In [2]:
result

tensor([ 0.0221,  0.0072,  0.0150,  ...,  0.0252, -0.0096,  0.0194],
       device='cuda:0')

In [4]:
import pandas as pd
file = pd.read_json('../LESS/LESS_output/CTA_SimTab/LESS_subset.jsonl',lines=True)
file

,dataset,id,messages
0,CTA_SimTab,CTA_SimTab_708,"[{'role': 'user', 'content': 'You are an exper..."
1,CTA_SimTab,CTA_SimTab_614,"[{'role': 'user', 'content': 'You are an exper..."
2,CTA_SimTab,CTA_SimTab_3391,"[{'role': 'user', 'content': 'You are an exper..."
3,CTA_SimTab,CTA_SimTab_3724,"[{'role': 'user', 'content': 'You are an exper..."
4,CTA_SimTab,CTA_SimTab_2936,"[{'role': 'user', 'content': 'You are an exper..."
...,...,...,...
987,CTA_WebTable,CTA_WebTable_12957,"[{'role': 'user', 'content': 'You are an exper..."
988,CTA_WebTable,CTA_WebTable_9556,"[{'role': 'user', 'content': 'You are an exper..."
989,CTA_SimTab,CTA_SimTab_941,"[{'role': 'user', 'content': 'You are an exper..."
990,CTA_SimTab,CTA_SimTab_1813,"[{'role': 'user', 'content': 'You are an exper..."


In [21]:
task = 'CTA'
dataset = 'SimTab'
pd.read_json('LESS_output/{task}_{dataset}/LESS_subset.jsonl',lines=True)
def content_extraction(row):
    instruction = row[-1][0]['content']
    input = ''
    output = row[-1][1]['content']
    return instruction,input,output
content_subset = file.apply(content_extraction,axis=1,result_type='expand')
content_subset.columns = ['instruction','input','output']
json.dump(content_subset.to_dict(orient='records'), open(f'../DataSelection-IF/train/{task}/{dataset}/train-select-w-LESS.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_1026305/2719693357.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  instruction = row[-1][0]['content']
/tmp/ipykernel_1026305/2719693357.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output = row[-1][1]['content']
